https://realpython.com/async-io-python/#setting-up-your-environment "Async IO Explained" ....


In [4]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1) #Nieblokujaca funkcja, ktora sygnalizuje ze kontrola moze byc przekazana z powrotem
                            # do 'event loop' czyli w tym przypadku maina
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())


import time
s = time.perf_counter()
await(main())
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
executed in 1.01 seconds.


# Próbujemy:

In [18]:
import asyncio
import time


async def wait_s(s: int) -> None:
    for _ in range(s):
        await asyncio.sleep(0.5)
        print("x")
    return 0

async def count_5():
    print("1")
    print("2")
    print("3")
    y= await wait_s(5)
    print("4")
    print("5")
    
async def count_d():
    for _ in range(5):
        await asyncio.sleep(0.5)
        print("y")
    return 0
    
    
async def main():
    await asyncio.gather(count_5(), count_d())
    
await(main())
    

1
2
3
x
y
x
y
x
y
x
y
x
4
5
y


In [27]:
import asyncio
import random

# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)

async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = random.randint(0, 10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(1)
        i = random.randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i

async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res

if __name__ == "__main__":
    random.seed(444)
    r1, r2, r3 = await(main())
    print()
    print(f"r1: {r1}, r2: {r2}, r3: {r3}")

Initiated makerandom(0).
makerandom(0) == 4 too low; retrying.
Initiated makerandom(1).
makerandom(1) == 4 too low; retrying.
Initiated makerandom(2).
makerandom(2) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 8 too low; retrying.
---> Finished: makerandom(2) == 10
makerandom(0) == 7 too low; retrying.
makerandom(1) == 8 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(0) == 1 too low; retrying.
makerandom(1) == 6 too low; retrying.
makerandom(0) == 9 too low; retrying.
makerandom(0) == 3 too low; retrying.
---> Finished: makerandom(1) == 9
makerandom(0) == 7 too low; retrying.
---> Finished: makerandom(0) == 10

r1: 10, r2: 9, r3: 10


# #####################DESIGN PATTERNS#######################

## Chaining Coroutines
A key feature of coroutines is that they can be chained together. (Remember, a coroutine object is awaitable, so another coroutine can await it.) This allows you to break programs into smaller, manageable, recyclable coroutines:

In [ ]:
import asyncio
import random
import time

async def part1(n: int) -> str:
    i = random.randint(0, 10)
    print(f"part1({n}) sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-1"
    print(f"Returning part1({n}) == {result}.")
    return result

async def part2(n: int, arg: str) -> str:
    i = random.randint(0, 10)
    print(f"part2{n, arg} sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-2 derived from {arg}"
    print(f"Returning part2{n, arg} == {result}.")
    return result

async def chain(n: int) -> None:
    start = time.perf_counter()
    p1 = await part1(n)
    p2 = await part2(n, p1)
    end = time.perf_counter() - start
    print(f"-->Chained result{n} => {p2} (took {end:0.2f} seconds).")

async def main(*args):
    await asyncio.gather(*(chain(n) for n in args))

if __name__ == "__main__":
    import sys
    random.seed(444)
    args = [1, 2, 3] if len(sys.argv) == 1 else map(int, sys.argv[1:])
    start = time.perf_counter()
    asyncio.run(main(*args)) ########################################## w przypadku jupytera trzeba zamienic na await
    end = time.perf_counter() - start
    print(f"Program finished in {end:0.2f} seconds.")

In [29]:
import asyncio
import itertools as it
import os
import random
import time

async def makeitem(size: int = 5) -> str:
    return os.urandom(size).hex()

async def randsleep(caller=None) -> None:
    i = random.randint(0, 10)
    if caller:
        print(f"{caller} sleeping for {i} seconds.")
    await asyncio.sleep(i)

async def produce(name: int, q: asyncio.Queue) -> None:
    n = random.randint(0, 10)
    for _ in it.repeat(None, n):  # Synchronous loop for each single producer
        await randsleep(caller=f"Producer {name}")
        i = await makeitem()
        t = time.perf_counter()
        await q.put((i, t))
        print(f"Producer {name} added <{i}> to queue.")

async def consume(name: int, q: asyncio.Queue) -> None:
    while True:
        await randsleep(caller=f"Consumer {name}")
        i, t = await q.get()
        now = time.perf_counter()
        print(f"Consumer {name} got element <{i}>"
              f" in {now-t:0.5f} seconds.")
        q.task_done()

async def main(nprod: int, ncon: int):
    q = asyncio.Queue()
    producers = [asyncio.create_task(produce(n, q)) for n in range(nprod)]
    consumers = [asyncio.create_task(consume(n, q)) for n in range(ncon)]
    await asyncio.gather(*producers)
    await q.join()  # Implicitly awaits consumers, too
    for c in consumers:
        c.cancel()

if __name__ == "__main__":
    import argparse
    random.seed(444)
    parser = argparse.ArgumentParser()
    parser.add_argument("-p", "--nprod", type=int, default=5)
    parser.add_argument("-c", "--ncon", type=int, default=10)
    ns = parser.parse_args()
    start = time.perf_counter()
    await (main(**ns.__dict__))
    elapsed = time.perf_counter() - start
    print(f"Program completed in {elapsed:0.5f} seconds.")

usage: ipykernel_launcher.py [-h] [-p NPROD] [-c NCON]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\MRatajczak\AppData\Roaming\jupyter\runtime\kernel-9ef29fba-a2cf-46de-be07-148a9e1504ce.json


SystemExit: 2